In [2]:
from similarity import euclid as euclid
import numpy as np

def _getApex(simplexMatrix,dists):
    '''Determines the Cartesian coordinates for the last vertex'''
    '''of a simplex given the distances of a point o to the reference points p1,...,pn used to'''
    '''generate the simplex represented by the simplexMatrix'''
    ''' INPUT:'''
    '''     *simplexMatrix: a simplex base matrix of the form'''
    '''           |0    0   ...           0|'''
    '''           |v21  0   ...           0|'''
    '''           |v31  v32 ...           0|\in R^n x R^(n-1)'''
    '''           |:    :   :             0|'''
    '''           |vn1  vn2 ...     vn(n-1)|'''
    '''  *dists: the distances between the point to be projected and each reference objects of the simplex base.'''
    '''           dists=[d1,...,dn] \in R^1 x R^n'''
    ''' OUTPUT'''
    ''' * newApex: the cartesian coordinates for the point o s.t. l2(newApex,simplexMatrix(i,:))=d(o,pi)'''
    '''            newApex \in \in R^1 x R^n;'''

    n = dists.shape[1]  # size(dists,2)
    newApex = np.zeros(1,n) 
    newApex[1] = dists[1]
    for k in range(2,n-1):  
        ldist = euclid(newApex[1], simplexMatrix[k,1]);  # both were :k-1
        d = dists(k)
        xN = simplexMatrix[k,k-1] # indices?
        yN = newApex[k-1]
        secondLastVal = yN - (d * d - ldist * ldist) / (2 * xN)
        if ~ np.isfinite(secondLastVal):
            secondLastVal = yN

        newApex(k-1)=secondLastVal
        lastVal = sqrt(yN * yN - secondLastVal * secondLastVal)  #  todo TO check
        if not np.isfinite(lastVal) or not np.isreal(lastVal):
            lastVal=0

        newApex[k]=lastVal
        return newApex


SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (3939055014.py, line 33)

In [3]:
class _NSimplex:
    # % Class to construct a (n-1)-dimensional simplex given n reference object p1,..,pn and to compute the n-Simplex projection of a point
    # %
    # % GENERAL INFO ON THE N-SIMPLEX PROJECTION
    # % If a metric space (X,d) meets the (n-1)-point property then for any n
    # % objects sampled from the original space, there exists an (n-1)-dimensional
    # % simplex in the Euclidean space whose edge lengths correspond to the 
    # % actual distances between the objects.
    # %
    # % Let SIGMA=<p1,..,pn>  the simplex generated by the point p1,..pn \in (X,d)
    # % and let v1,...,vn  the corresponding vertices, i.e. v1,...,vn are vectors in R^(n-1).
    # % s.t. l2(vi,vj)=d(pi,pj) \forall i,j=1..n
    # % We call "baseSimplex" the matrix  [v1;...;vn] (i.e. baseSimplex(i,:)=vi)
    # %
    # % Given the baseSimplex and a point o, and the distances d(o,pi) \forall i=1,..,n, 
    # % it is possible to project the point o into the point vo\in R^n s.t.
    # % l2(vo,vi)=d(o, pi) for all i
    # %
    # % Given two objects q and o, 
    # % a baseSimplex generated from <p1,..,pn>, 
    # % and the distances
    # % d(q,pi) for i=1,..,n, 
    # % d(o,pi) for i=1,..,n, 
    # % the projected points vo, vq satisfies
    # % l2(vq,vo)<=d(o,q)<=l2(vq,vo_minus)
    # % where 
    # % vo_minus= [vo(1:end-1),-vo(end)]
    # %
    # %NSimplex Properties:
    # % metric           - Distance used to compare two objects in X. It should be
    # %                    exends the MetricAbstract class, e.g. L2metric
    # %
    # % referenceObjects - nxD matrix containing the coordinates of n 
    # %                    reference objects (pivots) p1,..,pn \in X used to build 
    # %                    the baseSimplex 
    # %                           referenceObjects(i,:)=pi;
    # % 
    # % baseSimplex      - n x(n-1) strictly triangular matrix
    # %                    The rows are the coordinates of the vertices v1,...,vn 
    # %                    of the simplex built from the set of referenceObjects (i.e., p1,..,pn).
    # %                 
    # %                     e.g.  |0,   0   ...           0|
    # %                           |v21  0   ...           0|
    # %                           |v31  v32 ...           0|
    # %                           |:    :   :              |
    # %                           |vn1  vn2 ...     vn(n-1)|
    # %NSimplex Methods:
    # %
    # % NSimplex(referenceObjects, metric)- class constructor 
    # %                    INPUT:
    # %                     * referenceObjects: matrix containing the coordinates of n 
    # %                              reference objects(each row is an object)
    # %                     * metric:  the distance used to compare two objects
    # %                              (should exends the MetricAbstract class, e.g. L2metric)
    # % 
    # % getApexFromDists(dists) - Determines the Cartesian coordinates for the last vertex of a simplex,
    # %                 given only the distances (dists) between the point to be projected and each reference objects
    # % 
    # % project(point) - Use the "baseSimplex" to project the "point", i.e. determines the Cartesian coordinates 
    # %                   of the apex of an n-dimensional simplex determined given the coordinates of an (n - 1)-dimensional simplex
    # %                   and the distances from the new apex to each vertex in the existing simplex                
    # %
    # % see also https://arxiv.org/pdf/1707.08370.pdf
   
    # properties
    #     metric %Distance used to compare two objects in the original space
    #     referenceObjects %nxD matrix containing the coordinates of the n reference objects used to built teh simplex
    #     baseSimplex %n x(n-1)matrix representing a (n-1)-dimensional simplex 
    # end

def ___init___(self,referenceObjects, metric):
    '''referenceObjects: matrix containing the coordinates of n reference objects(each row is an object)'''
    '''metric:  the distance used to compare two objects'''

    self.metric = metric
    self.referenceObjects = referenceObjects
    n = referenceObjects.size    # check this WRONG
    self.baseSimplex = np.zeros(n,n-1)
    self.baseSimplex[2,1] = metric(referenceObjects(2),referenceObjects(1))
    if self.baseSimplex[2,1] == 0:
        raise Exception('zero distance points for base simplex')
        
    # for every further reference point, add the apex
    for k in range(3,n+1): # added one since Python end range non inclusive
        # distances od p_k to all previous points (p1, ..,p_(k-1)
        # RC: so dists_k is a single row, maybe make this a column...
        dists_k = np.zeros(1,k-1)
                
        for j in range(1,k): # was k -1 
            dists_k[j] = metric(referenceObjects(k), referenceObjects(j));

        if min(dists_k) == 0:
            raise Exception('zero distance points for base simplex!Bad choice for referenceObjects'); %TODO: skip thepivot and reduce the base 
                
        apex = _getApex(self.baseSimplex(1:k-1,1:k-2), dists_k) # indices not changed
        self.baseSimplex[k,1:k-1]= apex # indices not changed
            

def getApexFromDists(self, dists):
    '''Determines the Cartesian coordinates for the last vertice of a simplex'''
    '''given only the distances (dists) between the point to be projected and its'''
    '''distances to each vertex of the simplex base (i.e. the rows of the baseSimplex  matrix).'''
    return _getApex(self.baseSimplex,dists) # was NSimplexRc.

def project(self, point):
    '''Use the "baseSimplex" to project the "point"'''
    '''determines the Cartesian coordinates''' 
    '''of the apex of an n-dimensional simplex determined given the coordinates of an (n - 1)-dimensional simplex'''
    '''and the distances from the new apex to each vertex in the existing simplex,'''
    actual_metric=self.metric;
    dists_o= np.invert(actual_metric(point,self.referenceObjects))
    return _getApex(self.baseSimplex,dists_o)
     
def getReducedSimplex(this,k):
        reducedSimplex=this # what assignments/update sematics in both
        reducedSimplex.referenceObjects=this.referenceObjects[1:k] # indexing iffy!
        reducedSimplex.baseSimplex=this.baseSimplex[1:k, 1:k-1] # indexing iffy!
        return reducedSimplex

# NOT USED CURRENTLY DO LATER:   
# function newApex=getExtendedApexGivenDist(obj,oldApex,distsLastPivots)
#           old_n=size(oldApex,2);
#           n=size(distsLastPivots,2);
#           newApex=zeros(1,n+old_n);
          
#           newApex(1:old_n)=oldApex;
#           simplexMatrix=obj.baseSimplex;
#           for k=old_n+1:old_n+n
#               ldist=euc(newApex(1:k-1), simplexMatrix(k,1:k-1)); %here size(newApex)=(1,k-1)
#               d = distsLastPivots(k-old_n);
#               xN = simplexMatrix(k,k-1);
#               yN = newApex(k-1);
#               secondLastVal = yN - (d * d - ldist * ldist) / (2 * xN);
#               if(~isfinite(secondLastVal))
#                   secondLastVal=yN;
#               end
#               newApex(k-1)=secondLastVal;
#               lastVal = sqrt(yN * yN - secondLastVal * secondLastVal);% todo TO check
#               if(~isfinite(lastVal)|| ~isreal(lastVal))
#                   lastVal=0;
#               end
#               %                 else
#               %                     if (abs(lastVal)<10^-5)
#               %                         lastVal=abs(lastVal);
#               %                     end
#               %                     if (lastVal<0)
#               %                         %lastVal=abs(lastVal);
#               %                          error('Error: set of points that not satisfies the n-point property');
#               %                     end
#               newApex(k)=lastVal;
#           end
#       end %getExtendedApexGivenDist
#   %%     
#   function newApex=getExtendedApex(obj,apex,o,n)
#       newApex=zeros(1,n);
#       old_n=size(apex,2);
#       newApex(1:old_n)=apex;
#       actual_metric=obj.metric;
#       dists=actual_metric.distanceToMorePoints(o,obj.referenceObjects);
#       simplexMatrix=obj.baseSimplex;
#       for k=old_n+1:n
#           ldist=euc(newApex(1:k-1), simplexMatrix(k,1:k-1)); %here size(newApex)=(1,k-1)
#           d = dists(k);
#           xN = simplexMatrix(k,k-1);
#           yN = newApex(k-1);
#           secondLastVal = yN - (d * d - ldist * ldist) / (2 * xN);
#           if(~isfinite(secondLastVal))
#               secondLastVal=yN;
#           end
#           newApex(k-1)=secondLastVal;
#           lastVal = sqrt(yN * yN - secondLastVal * secondLastVal);% todo TO check
#           if(~isfinite(lastVal)|| ~isreal(lastVal))
#               lastVal=0;
#           end
#           %                 else
#           %                     if (abs(lastVal)<10^-5)
#           %                         lastVal=abs(lastVal);
#           %                     end
#           %                     if (lastVal<0)
#           %                         %lastVal=abs(lastVal);
#           %                          error('Error: set of points that not satisfies the n-point property');
#           %                     end
#           newApex(k)=lastVal;
#       end
#   end %getApex 


IndentationError: expected an indented block after class definition on line 1 (2022827344.py, line 71)

In [4]:
class NsimpProjection:
    def ___init___(self,witnessData, projectDims, metric):
        '''An NsimplexProjection'''
        '''witnessData is a numpy array - witnesses are drawn from this must be at least projectDims in size'''
        '''projectDims is the number of dimensions of the simplex'''
        '''metric is a true metric'''

        self.projectDims = projectDims
        self.nSimplex = _NSimplex(witnessData[:projectDims],metric)

    
    def project(self,A):
        return self.nSimplex.project(A)

    def projectWithDistances(self,qToLDists):
        return np.invert(self.nSimplex.getApexFromDists(qToLDists[1:self.projectDims]))


## Test code
What tests can we do?

witness_data = 0  # need a NP array
simp = NsimpProjection(witness_data,2,euclid)